### FoT データを、画像検索用のデータセットにサンプリングする。  
  
#### FoTデータの場所:  
/mnt/mnt/nfs/dataset/dev_FOT_data_yachide/Classification_MOVIE  
  
318029動画ある  
1動画およそ1分  
10fps  
640 x 340  
  
#### 元データのディレクトリ構成は以下:  
Classification_MOVIE  
　　　　|- DAYTIME_FINE_JP_HIGHWAY  
　　　　|　　　　|-{purpose}_{バージョン}_{トリップ開始時刻}  
　　　　|　　　　|　　　　LMOVIE  
　　　　|　　　　|　　　　　　|-{動画開始時刻}_{動画中央時刻}_{動画終了時刻}.mp4  
　　　　|　　　　|　　　　　　...  
　　　　|　　　　...  
　　　　|  
　　　　|- {DAY/NIGHT}_{WEATHER}_{JAPAN/NA}_{HIGHWAY/NORMAL}  
　　　　...       
  
#### 以下のディレクトリに出力したい:  
/mnt/mnt/nfs/home/yasumasa_kobayashi/codes/lemniscate/datasets/data/FoT_yachide  
  
サンプリング後のディレクトリ構成を以下としたい:  
FoT_yachide  
　　　　|- train  
　　　　|　　　　|-{purpose}_{バージョン}_{トリップ開始時刻}_{動画開始時刻}_{動画中央時刻}_{動画終了時刻}　★  
　　　　|　　　　|　　　　|- {purpose}_{バージョン}_{トリップ開始時刻}_{動画開始時刻}_{動画中央時刻}_{動画終了時刻}_train_001.jpeg  
　　　　|　　　　|　　　　|- {purpose}_{バージョン}_{トリップ開始時刻}_{動画開始時刻}_{動画中央時刻}_{動画終了時刻}_train_002.jpeg  
　　　　|　　　　|　　　　|- {purpose}_{バージョン}_{トリップ開始時刻}_{動画開始時刻}_{動画中央時刻}_{動画終了時刻}_train_{連番3桁}.jpeg    
　　　　|　　　　|　　　　...  
　　　　|  　　　...
　　　　|  
　　　　L val  
　　　　　　　　|-{purpose}_{バージョン}_{トリップ開始時刻}_{動画開始時刻}_{動画中央時刻}_{動画終了時刻}　★  
　　　　　　　　|　　　　|- {purpose}_{バージョン}_{トリップ開始時刻}_{動画開始時刻}_{動画中央時刻}_{動画終了時刻}_val_001.jpeg  
　　　　　　　　|　　　　|- {purpose}_{バージョン}_{トリップ開始時刻}_{動画開始時刻}_{動画中央時刻}_{動画終了時刻}_val_002.jpeg  
　　　　　　　　|　　　　|- {purpose}_{バージョン}_{トリップ開始時刻}_{動画開始時刻}_{動画中央時刻}_{動画終了時刻}_val_{連番3桁}.jpeg  
　　　　　　　　|　　　　...  
　　　　  　　　...  
  
pytorchのfolder instanceで読み込むと、★がクラス(ID)になる i.e. フォルダ直下の画像が同じクラスとして扱われる  
  
#### サンプリング  
- 参考文献(Driving Scene-Retrieval by Example from Large-Scale Data)では、30fps (BDD) → 6fpsにしていた  
- とりあえず、1fpsでサンプリングする
- どれくらい学習に時間がかかるかを調べるために、3フレーム分投入する
    - 多分、resnetの最終出力ｘ３をconcatしているのでは？  
    
#### ざっくりとしたアルゴリズム  
- やりたいこと  
1. 動画ごとに、ディレクトリを作成し、動画をサンプリング。jpegとして格納  
　　・その際に、trainとvalをわけて格納する  
　　・step1としては、先頭3フレームをtrain、次の3フレームをvalにする。(1fpsとする)  
　　　・後々、フレーム数でどれくらい検索性能が変わるかを見たい場合は、1フレームずつサンプリングし、コピーするようにする  

・globで全データのmp4を取得する  
・先頭から  
　・ディレクトリを作成(train, val)  
　・サンプリングする(train, val)  
　・ディレクトリ内に格納する  
 

In [3]:
import glob
import os
import cv2
from tqdm import tqdm

FoT_directory =  '/mnt/mnt/nfs/dataset/dev_FOT_data_yachide/Classification_MOVIE'
list_of_mp4_files = glob.glob(FoT_directory+'/**/**.mp4',recursive=True)
print(len(list_of_mp4_files))

train_directory = '/mnt/mnt/nfs/home/yasumasa_kobayashi/codes/lemniscate/datasets/data/FoT_yachide/train'
val_directory = '/mnt/mnt/nfs/home/yasumasa_kobayashi/codes/lemniscate/datasets/data/FoT_yachide/val'

if not os.path.isdir(train_directory):
    os.mkdir(train_directory)
    os.chmod(train_directory, 0o777)
if not os.path.isdir(val_directory):
    os.mkdir(val_directory)
    os.chmod(val_directory, 0o777)

def save_frame_range(video_path, start_frame, end_frame, step_frame, save_path, train=True):
    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        return
    idx=0
    for i in range(start_frame, end_frame, step_frame):
        cap.set(cv2.CAP_PROP_POS_FRAMES, i)
        ret, frame = cap.read()
        if ret:
            idx+=1
            if train:
                save_name = save_path+'_train_'+str(idx).zfill(3)+'.jpeg'
            else:
                save_name = save_path+'_val_'+str(idx).zfill(3)+'.jpeg'
            cv2.imwrite(save_name, frame)
        else:
            return

def sample_video_frames(list_of_mp4_files):
    print('sampling video...', len(list_of_mp4_files))
    for i, mp4_file in enumerate(list_of_mp4_files):
    #for i, mp4_file in tqdm(enumerate(list_of_mp4_files)):
        #if i==10:
        #    break

        #ディレクトリを作成
        splitted_list = mp4_file.split('/')
        name_dir = splitted_list[8]+'_'+splitted_list[10][:-4] #{purpose}_{バージョン}_{トリップ開始時刻}_{動画開始時刻}_{動画中央時刻}_{動画終了時刻}
        train_dir = os.path.join(train_directory, name_dir)
        val_dir = os.path.join(val_directory, name_dir)
        if not os.path.isdir(train_dir):
            os.mkdir(train_dir)
            os.chmod(train_dir, 0o777)
        if not os.path.isdir(val_dir):
            os.mkdir(val_dir)
            os.chmod(val_dir, 0o777)

        #サンプリングをし、ディレクトリに保存する
        save_path = os.path.join(train_dir, name_dir) 
        save_frame_range(mp4_file, 0, 30, 10, save_path, train=True)
        save_path = os.path.join(val_dir, name_dir) 
        save_frame_range(mp4_file, 30, 60, 10, save_path, train=False)
    
### 並列処理で実行
#from multiprocessing import Pool
#pool = Pool(processes = 40)
#pool.map(sample_video_frames(list_of_mp4_files))
#pool.close()

import threading

class Sampling_Thread(threading.Thread):

    def __init__(self, thread_name, list_of_mp4_files):
        self.thread_name = str(thread_name)
        threading.Thread.__init__(self)
        self.list_of_mp4_files = list_of_mp4_files

    def __str__(self):
        return self.thread_name

    def run(self):
        print('Thread: %s started.' % self.thread_name)
        print('size: ',len(self.list_of_mp4_files))
        sample_video_frames(self.list_of_mp4_files)
        print('Thread: %s ended.' % self.thread_name)
        
thread_list = []
for i in range(len(list_of_mp4_files)//10000+1):
    #print(len(list_of_mp4_files[i*10000:(i+1)*10000]))
    _sub = list_of_mp4_files[i*10000:(i+1)*10000]
    thread = Sampling_Thread(thread_name=i, list_of_mp4_files=_sub)
    thread.start()
    thread_list.append(thread)

for thread in thread_list:
    thread.join()

print('finished')


    
    

318029
Thread: 0 started.
size:  10000
sampling video... 10000
Thread: 1 started.
size:  10000
sampling video... 10000
Thread: 2 started.
size:  10000
sampling video... 10000
Thread: 3 started.
size:  10000
sampling video... 10000
Thread: 4 started.
size:  10000
sampling video... 10000
Thread: 5 started.
size:  10000
sampling video... 10000
Thread: 6 started.
size:  10000
sampling video... 10000
Thread: 7 started.
size:  10000
sampling video... 10000
Thread: 8 started.
size:  10000
sampling video... 10000
Thread: 9 started.
size:  10000
sampling video... 10000
Thread: 10 started.
size:  10000
sampling video... 10000
Thread: 11 started.
size:  10000
sampling video... 10000
Thread: 12 started.
size:  10000
sampling video... 10000
Thread: 13 started.
size:  10000
sampling video... 10000
Thread: 14 started.
size:  Thread: 15 started.10000
sampling video... 10000

size:  10000Thread: 16 started.
Thread: 17 started.
size:  10000
sampling video... 10000
sampling video... 10000

Thread: 18 sta

In [46]:
import os
name = '/mnt/mnt/nfs/dataset/dev_FOT_data_yachide/Classification_MOVIE/DAYTIME_FINE_JP_HIGHWAY/ASP_20160830001_1470800158778/MOVIE/1470806999691_1470807029691_1470807059691.mp4'
splitted_list = name.split('/')
print(splitted_list)
name_dir = splitted_list[8]+'_'+splitted_list[10][:-4]
print(name_dir)
train_save_name = name_dir + '_train_001.jpeg'
train_save_name = os.path.join(name_dir, train_save_name)
print(train_save_name)


['', 'mnt', 'mnt', 'nfs', 'dataset', 'dev_FOT_data_yachide', 'Classification_MOVIE', 'DAYTIME_FINE_JP_HIGHWAY', 'ASP_20160830001_1470800158778', 'MOVIE', '1470806999691_1470807029691_1470807059691.mp4']
ASP_20160830001_1470800158778_1470806999691_1470807029691_1470807059691
ASP_20160830001_1470800158778_1470806999691_1470807029691_1470807059691/ASP_20160830001_1470800158778_1470806999691_1470807029691_1470807059691_train_001.jpeg


In [1]:
import glob
FoT_directory =  '/mnt/mnt/nfs/dataset/dev_FOT_data_yachide/Classification_MOVIE'
list_of_mp4_files = glob.glob(FoT_directory+'/**/**.mp4',recursive=True)
data_len = len(list_of_mp4_files)
_steps = data_len/40
i = range(0, data_len, int(_steps+1))
print(i[0], i[1])    

0 7951


In [2]:
for i in range(len(list_of_mp4_files)//10000+1):
    _sub = list_of_mp4_files[i*10000:(i+1)*10000]
    print(len(list_of_mp4_files[i*10000:(i+1)*10000]))

10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
10000
8029
